# **PRÁCTICA FINAL - PREDICCIÓN DE DURACIÓN DE VIAJES Y DETECCIÓN DE EMERGECIAS EN TWEETS**

**Asignatura:** Modelos no supervisados

**Fecha:** 09/06/2023

**Autores:** Mencía de Parias y Juan María Rotaeche

#### **PARTE 0: Librerías**

In [49]:
import pandas as pd
import warnings

from scipy import stats
from sklearn.preprocessing import PowerTransformer

warnings.filterwarnings('ignore')


### **PARTE 1: Predicción de duración de viajes**

##### **Ejercicio 1**

Realizar preprocesamiento de datos: imputar valores faltantes, transformar variables categóricas, estandarizar variables numéricas, etc. 
si lo consideras necesario para futuros modelos. Puede ser interesante intentar adaptar las variables que no siguan una distribución normal mediante técnicas de mapeado a gausianas como Power Transformers (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html#sklearn.preprocessing.PowerTransformer). *(2puntos)*

In [2]:
uber_examples = pd.read_csv('uber_time_examples.csv')
uber_labels = pd.read_csv('uber_time_labels.csv')

In [14]:
print (uber_examples.shape, 
       uber_labels.shape)
print (uber_examples.head(), 
       uber_labels.head())

(400000, 12) (400000, 2)
   id       feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0   0  01-07 17:04:08          2       1.20        263        141  12.513054   
1   1  03-02 17:41:40          1       0.88        246         68   6.256527   
2   2  02-17 12:15:00          3       7.61         24         13  18.769581   
3   3  03-30 13:59:42          1       1.50        239        163   6.256527   
4   4  02-14 18:26:55          1       1.20        142        229   6.256527   

    feature_6  feature_7  feature_8  feature_9  feature_10  
0  297.430685  56.317405     405.20   0.408689  126.689773  
1  278.205127  27.160167     314.88  -0.256911  126.693467  
2   27.141964   5.192385      44.61  56.880789  126.615789  
3  270.288721  65.104518     403.50   1.218689  126.686311  
4  160.589952  91.465857     372.20   0.408689  126.689773      id  duration
0   0     455.0
1   1     413.0
2   2    1501.0
3   3     514.0
4   4     605.0


Ambas tablas tienen el mismo número de filas y la columna *id* para identificar los registros, por lo que el primer paso es unir los datos en base a esta columna.

In [18]:
uber = pd.merge (uber_labels, uber_examples, on = 'id')
# uber.set_index('id', inplace= True)
uber.head()

,id,duration,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10
0,0,455.0,01-07 17:04:08,2,1.20,263,141,12.513054,297.430685,56.317405,405.20,0.408689,126.689773
1,1,413.0,03-02 17:41:40,1,0.88,246,68,6.256527,278.205127,27.160167,314.88,-0.256911,126.693467
2,2,1501.0,02-17 12:15:00,3,7.61,24,13,18.769581,27.141964,5.192385,44.61,56.880789,126.615789
3,3,514.0,03-30 13:59:42,1,1.50,239,163,6.256527,270.288721,65.104518,403.50,1.218689,126.686311
4,4,605.0,02-14 18:26:55,1,1.20,142,229,6.256527,160.589952,91.465857,372.20,0.408689,126.689773


Una vez tenemos los datos listos para empezar a trabajar, primero vamos a realizar un preprocesamiento de datos. Para ello, primero trabajaremos con los valores nulos si los hubiera; después analizaremos el tipo de variables que tenemos y si es necesario transformar alguna; y si realizamos una transformación a las variables que no siguen una distribución normal.

In [19]:
uber.isnull().sum()

id            0
duration      0
feature_0     0
feature_1     0
feature_2     0
feature_3     0
feature_4     0
feature_5     0
feature_6     0
feature_7     0
feature_8     0
feature_9     0
feature_10    0
dtype: int64

En todo nuestro conjunto de datos no hay ningún valor faltante.

In [21]:
uber.dtypes

id              int64
duration      float64
feature_0      object
feature_1       int64
feature_2     float64
feature_3       int64
feature_4       int64
feature_5     float64
feature_6     float64
feature_7     float64
feature_8     float64
feature_9     float64
feature_10    float64
dtype: object

In [24]:
uber['feature_0'] = pd.to_datetime(uber['feature_0'], format='%m-%y %H:%M:%S')

In [43]:
for columna in uber.columns:
    datos = uber[columna].values
    stat, p_valor = stats.shapiro(datos) # Prueba de Shapiro-Wilk
    alpha = 0.05 # Comprobar la normalidad
    if p_valor > alpha:
        print(f"La variable '{columna}' sigue una distribución normal.")
    else:
        print(f"La variable '{columna}' no sigue una distribución normal.")

c:\Users\menci\anaconda3\lib\site-packages\scipy\stats\morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


La variable 'id' no sigue una distribución normal.
La variable 'duration' no sigue una distribución normal.
La variable 'feature_0' no sigue una distribución normal.
La variable 'feature_1' no sigue una distribución normal.
La variable 'feature_2' no sigue una distribución normal.
La variable 'feature_3' no sigue una distribución normal.
La variable 'feature_4' no sigue una distribución normal.
La variable 'feature_5' no sigue una distribución normal.
La variable 'feature_6' no sigue una distribución normal.
La variable 'feature_7' no sigue una distribución normal.
La variable 'feature_8' no sigue una distribución normal.
La variable 'feature_9' no sigue una distribución normal.
La variable 'feature_10' no sigue una distribución normal.


In [50]:
columnas_transformar = ['feature_1','feature_2','feature_3','feature_4','feature_5','feature_6','feature_7','feature_8','feature_9','feature_10']

transformador = PowerTransformer(method='yeo-johnson')

uber[columnas_transformar] = transformador.fit_transform(uber[columnas_transformar])

La única columna a la que conviene cambiarle el tipo de datos es a *feature_0* ya que muestra una fecha pero está en formato object.

Hemos realizado la prueba de Shapiro-Wilk para comprobar qué variables no siguen una distribución normal, y posteriormente hemos normalizado todas las variables excepto la fecha y el id con el método Yeo-Johnson de PowerTransformer.

##### **Ejercicio 2**

Crear nuevas características (features) que puedan mejorar el poder predictivo del modelo. *(1 punto)*

In [51]:
uber.nunique() #ver si es conveniente poner feature_1 y feature_5 como dummies

id            400000
duration        5592
feature_0     387400
feature_1         10
feature_2       2709
feature_3        231
feature_4        255
feature_5         10
feature_6        231
feature_7        255
feature_8      39968
feature_9       2709
feature_10      2709
dtype: int64

##### **Ejercicio 3**

Seleccionar las características más relevantes para predecir la duración del viaje. Utilizar técnicas de selección de características basadas en una sola variable o SelectFromModel. Evitar Recursive feature elimination debido a su alto coste computacional.
Ver : (https://scikit-learn.org/stable/modules/feature_selection.html) *(2puntos)*

##### **Ejercicio 4**

Entrenar un modelo sencillo como base y medir su MAPE (Mean Absolute Percentage Error) en el conjunto de test. Luego, elegir y entrenar dos modelos más avanzados (por ejemplo, ensambladores, máquinas de soporte vectorial, modelos bayesianos, redes neuronales) y comparar sus MAPEs. *(2puntos)*

##### **Ejercicio 5**

Optimizar los hiperparámetros de los dos últimos modelos utilizando validación cruzada (cross-validation) y comparar sus MAPEs. Elegir el mejor modelo basándose en estos resultados. *(2puntos)*

### **PARTE 2: Detección de emergencias en tweets**

##### **Ejercicio 1:**

Extraer los embeddings del texto de los tweets utilizando un modelo pre-entrenado de Huggingface. *(1.5 puntos)*

##### **Ejercicio 2:**

Crear y entrenar una pequeña red neuronal que utilice los embeddings, la palabra clave (keyword) y la ubicación (location) para predecir si un tweet está relacionado con una emergencia o no. Gestionar los valores faltantes y agrupar las variables categóricas de manera adecuada. No es necesario realizar una optimización de hiperparámetros exhaustiva, pero se pueden realizar ajustes si se desea. 

NOTA: Si no se ha podido calcular los embeddings del ejercicio anterior, usar los que aparecen guardados como numpy.array en el fichero. *(1.5 puntos)*